# ETL Lab

### Introduction

In this lab, we ask you to use the techniques learned in this section to work with an API of your choosing.  As important to getting to the correct code is to develop the proper procedure for getting there.  Just like in the preceding lessons we will following our procedures such as: 

1. Red, green, refactor
2. Move mess into an object
3. Make small methods by: 
    A. Commenting code
    B. Translating comments into methods
    
Along the way, we will arrive at our pattern of a *Client*, *Adapter*, and *Target*.

### Step 1.  Just get the data

The first step is to go from red to green.  That is, the code starts off with nothing working and our task is simply to get it working.  In this case, this means the following: 

1. Call an API of your choosing
2. Return a list of dictionaries and store as a variable named `entities`

In [6]:
import requests
response = requests.get('https://rickandmortyapi.com/api/character/?status=alive')
entities = response.json()['results']


In [8]:
type(entities)
# list 

type(entities[0])
# dict

dict

### Step 2. Change the dictionaries into objects

The next step is to change dictionaries received back from the API into objects.  We can break this down into a couple of steps.

1. Create the *target class*.  This is the class the dictionaries will be transformed into.  To do this, choose no more than five attributes to store in each instance.

In [17]:
entities[8]

{'id': 24,
 'name': 'Armagheadon',
 'status': 'Alive',
 'species': 'Alien',
 'type': 'Cromulon',
 'gender': 'Male',
 'origin': {'name': 'Signus 5 Expanse',
  'url': 'https://rickandmortyapi.com/api/location/22'},
 'location': {'name': 'Signus 5 Expanse',
  'url': 'https://rickandmortyapi.com/api/location/22'},
 'image': 'https://rickandmortyapi.com/api/character/avatar/24.jpeg',
 'episode': ['https://rickandmortyapi.com/api/episode/16'],
 'url': 'https://rickandmortyapi.com/api/character/24',
 'created': '2017-11-05T08:48:30.776Z'}

In [12]:
class Character:
    # please change the name of this class
    def __init__(self,name,species,gender,location):
        self.name = name
        self.species = species
        self.gender = gender
        self.location = location

Check your work by assigning an instance to the variable `target_instance`.

In [18]:
target_instance = Character('Armagheadon', 'Alien', 'Male', 'Signus 5 Expanse')
3 < len( target_instance.__dict__.keys()) < 5
# True 

True

1. Reject some of the data

We don't want to pass all of our data into our class.  So create a smaller dictionary of just the attributes we need.

In [19]:
selected_attributes = {'name':'Armagheadon', 'species':'Alien', 'gender':'Male', 'location': 'Signus 5 Expanse' }

In [20]:
type(selected_attributes)
# dict

dict

In [21]:
len(selected_attributes.keys()) == len(target_instance.__dict__.keys())
# True

True

2. Coerce dictionaries into objects

A. To start, coerce just one dictionary into an object.

In [22]:
first_entity = entities[0]
first_attributes = {'name': first_entity['name'], 'species': first_entity['species'],
                    'gender':first_entity['gender'], 'location': first_entity['location']['name']}
first_object = Character(first_attributes['name'], first_attributes['species'], 
                         first_attributes['gender'], first_attributes['location'] )
# change the above line to reference your target class

In [23]:
list(first_object.__dict__.values()) == list(entities[0].values())
# True

False

In [24]:
first_object.__dict__

{'name': 'Rick Sanchez',
 'species': 'Human',
 'gender': 'Male',
 'location': 'Earth (Replacement Dimension)'}

B. Now that you have solved for one, solve for all.  Coerce all of the dictionaries into objects.  Assign the list of objects to a variable `targets`.

In [25]:
targets = []
for entity in entities:
    attributes = {'name': entity['name'], 'species': entity['species'],
                    'gender':entity['gender'], 'location': entity['location']['name']}
    current_character = Character(attributes['name'], attributes['species'], 
                         attributes['gender'], attributes['location'] )
    targets.append(current_character)


In [26]:
len(targets) == len(entities)
# True

True

### 3. Move the remaining code into an object 

At this point, we successfully have transformed a list of dictionaries from an API into a list of objects.  But we need to keep cleaning up our code.  To do this, look at the code outside of a class, and move it into a class, with a method name of run.

In [50]:
class CharacterAdapter:
    # please change the name of this class     
    def run(self):
        response = requests.get('https://rickandmortyapi.com/api/character/?status=alive')
        entities = response.json()['results']
        targets = []
        for entity in entities:
            attributes = {'name': entity['name'], 'species': entity['species'],
                    'gender':entity['gender'], 'location': entity['location']['name']}
            current_character = Character(attributes['name'], attributes['species'], 
                         attributes['gender'], attributes['location'] )
            targets.append(current_character)
        return targets

Let's marke sure that this works.

In [51]:
adapter = CharacterAdapter()
# change the above line to reference your adapter
results = adapter.run()

In [52]:
len(results) == len(targets)
# True

True

### 4. Make the methods smaller

Next, separate out the run method in the adapter class smaller.  Do this by first writing comments in the code, and then moving the code into separate methods.  Please leave the comments in your code.  Your methods should be no longer than five lines long, and there can only be a total of one `if else` statement or `loop` per method.  Having both an `if else` and a `loop` in any method is also too complicated -- don't do it.

In [53]:
class CharacterAdapter:
    def run(self):
        # get data
        character_data = self.get_data()
        # get list of characters
        characters = self.get_characters(character_data)
        return characters
    def get_data(self):
        response = requests.get('https://rickandmortyapi.com/api/character/?status=alive')
        entities = response.json()['results']
        return entities
    
    def get_characters(self, entities):
        targets = []
        for entity in entities:
            attributes = {'name': entity['name'], 'species': entity['species'],
                    'gender':entity['gender'], 'location': entity['location']['name']}
            current_character = Character(attributes['name'], attributes['species'], 
                         attributes['gender'], attributes['location'] )
            targets.append(current_character)
        return targets

In [54]:
refactored_adapter = CharacterAdapter()
# change the above line to reference your adapter
refactored_results = adapter.run()


In [55]:
len(results) == len(targets) == len(refactored_results)


True

### 5. Create the client class

Next move calls to the API into their own separate class.  This way we can call the API but later to decide to coerce the data into different types of objects than we did above.

In [64]:
class Client:
    def run(self):
        response = requests.get('https://rickandmortyapi.com/api/character/?status=alive')
        entities = response.json()['results']
        return entities

Place the updated Adapter class below.  Check that it still works as it did before.

In [65]:
class CharacterAdapter:
    def run(self):
        
        # get data
        character_data = Client().run()
        # get list of characters
        characters = self.get_characters(character_data)
        return characters
    
    def get_characters(self, entities):
        targets = []
        for entity in entities:
            attributes = {'name': entity['name'], 'species': entity['species'],
                    'gender':entity['gender'], 'location': entity['location']['name']}
            current_character = Character(attributes['name'], attributes['species'], 
                         attributes['gender'], attributes['location'] )
            targets.append(current_character)
        return targets

In [66]:
refactored_adapter = CharacterAdapter()

len(refactored_adapter.run()) == len(adapter.run())

True

### Summary

Great job!  Hopefully, you saw how by building our code and then slowly refactoring our code, we can eventually get to some clean code.